In [1]:
import requests

# Vos identifiants client
CLIENT_ID = 'PAR_matchingesco_dcf45d374cd576dc2fddf2fcf68acbd09cc22cb453e770714c24bd1b0ee87557'
CLIENT_SECRET = 'bb844e4e645100d85efec1b83fa1e30e900f99692f55278873289f1aaa4affb4'
TOKEN_URL = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"  # Assurez-vous que l'URL est correcte
REGION_CODE = 53
MAX_OFFRES = 50

def get_access_token():
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'client_credentials',  # Spécifie le type d'accès
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    
    response = requests.post(TOKEN_URL, headers=headers, data=data)
    
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("Jeton d'accès obtenu:", access_token)
        return access_token
    else:
        print("Erreur lors de l'obtention du jeton d'accès:", response.status_code, response.text)
        return None


In [2]:
access_token = get_access_token()
access_token

Jeton d'accès obtenu: WoHQwo1Li3nXw-AZ4r5yVYM7NAM


'WoHQwo1Li3nXw-AZ4r5yVYM7NAM'

In [33]:
import schedule
import time
from datetime import datetime, timedelta
import pandas as pd

ENDPOINT = "https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
# Format de date souhaité
date_format = "%Y-%m-%dT%H:%M:%SZ"
# Date actuelle (aujourd'hui) au format spécifié
maxCreationDat = datetime.utcnow().strftime(date_format)
# Date d'hier (aujourd'hui - 1 jour) au format spécifié
minCreationDat = (datetime.utcnow() - timedelta(days=1)).strftime(date_format)

results_day = []

def fetch_offres_bretagne():
    # Obtenir le jeton d'accès
   
    access_token = get_access_token()

    if not access_token:
        return "Impossible de récupérer le jeton d'accès. Abandon de la requête."
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'region': REGION_CODE,
        'range': f'0-{150 - 1}', #Pagination des données. La plage de résultats est limitée à 150.
        'maxCreationDate' : maxCreationDat,
        'minCreationDate': minCreationDat
    }

    # Requête pour récupérer les offres
    response = requests.get(ENDPOINT, headers=headers, params=params)
    if response.status_code >= 200 and response.status_code < 300 :
        data = response.json()
        offres =  data["resultats"]
        print(f"[{datetime.now()}] Extraction réussie : {len(offres)} offres récupérées.")

        for offre in offres:
            results_day.append({'id' : offre['id'],
                'intitule' : offre['intitule'],
                'description' : offre['description'],
                'date_actu' : offre['dateActualisation'],
                'lieu_travail' : offre['lieuTravail'],
                'code_rome' : offre['romeCode']
            })    

        return pd.DataFrame(results_day)
    else:
        print(f"[{datetime.now()}] Échec de l'extraction. Code erreur : {response.status_code}, {response.text}")

#schedule.every().day.do(fetch_offres_bretagne)
#while True:
#    schedule.run_pending()
#    time.sleep(1)

In [34]:
results_day = fetch_offres_bretagne()
print(f"Nbre d'offres, {len(results_day)}")
results_day

Jeton d'accès obtenu: pHzyKoevYktVnkDU-jM84GTszP8
[2024-11-10 18:48:54.142406] Extraction réussie : 150 offres récupérées.
Nbre d'offres, 150


,id,intitule,description,date_actu,lieu_travail,code_rome
0,183YBBL,Agents d'étiquetage à domicile (H/F),"En tant qu'agent d'étiquetage à domicile, vous...",2024-11-10T15:11:49.242Z,"{'libelle': '56 - Arradon', 'latitude': 47.633...",K1304
1,183YBBJ,Agents d'étiquetage à domicile (H/F),"En tant qu'agent d'étiquetage à domicile, vous...",2024-11-10T15:11:13.069Z,"{'libelle': '56 - Arradon', 'latitude': 47.633...",K1304
2,183XZYR,Agents d'étiquetage à domicile (H/F),"En tant qu'agent d'étiquetage à domicile, vous...",2024-11-10T14:48:21.185Z,"{'libelle': '56 - Riantec', 'latitude': 47.722...",K1304
3,183XZYG,Agents d'étiquetage à domicile (H/F),"En tant qu'agent d'étiquetage à domicile, vous...",2024-11-10T14:42:40.057Z,"{'libelle': '56 - Grand-Champ', 'latitude': 47...",K1304
4,183XZYC,Agents d'étiquetage à domicile (H/F),"En tant qu'agent d'étiquetage à domicile, vous...",2024-11-10T14:41:30.798Z,"{'libelle': '56 - Plouay', 'latitude': 47.9322...",K1304
...,...,...,...,...,...,...
145,7345371,Technicien de maintenance F/H (H/F),Expert des technologies des énergies et de l'i...,2024-11-10T05:52:34.000Z,{'libelle': '29 - Finistère'},I1203
146,7345065,Technico-commercial F/H (H/F),L'ENTREPRISE VINCI Energies : une organisatio...,2024-11-10T05:51:36.000Z,{'libelle': '35 - Ille et Vilaine'},M1707
147,7345056,Monteur Sprinkler F/H (H/F),VINCI Energies : une organisation décentralis...,2024-11-10T05:51:34.000Z,{'libelle': '35 - Ille et Vilaine'},F1502
148,7344859,Technicien de maintenance CFO CFA (29) F/H (H/F),"VINCI Facilities Bretagne (7M€ de CA, 45 colla...",2024-11-10T05:50:55.000Z,{'libelle': '29 - Finistère'},I1203
